In [1]:
# import findspark
# findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("chapter-19-perf")\
    .getOrCreate()

SPARK_BOOK_DATA_PATH = "/home/gong/spark/books/Spark-The-Definitive-Guide"

In [5]:
file_path = SPARK_BOOK_DATA_PATH + "/data/flight-data/csv/2015-summary.csv"
# Original loading code that does *not* cache DataFrame
DF1 = spark.read.format("csv")\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .load(file_path)

In [6]:
DF1.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [7]:
%%time
DF2 = DF1.groupBy("DEST_COUNTRY_NAME").count().collect()

CPU times: user 11.2 ms, sys: 2.36 ms, total: 13.5 ms
Wall time: 2.04 s


In [8]:
%%time
DF3 = DF1.groupBy("ORIGIN_COUNTRY_NAME").count().collect()

CPU times: user 6.67 ms, sys: 0 ns, total: 6.67 ms
Wall time: 885 ms


In [11]:
DF3

[Row(ORIGIN_COUNTRY_NAME='Paraguay', count=1),
 Row(ORIGIN_COUNTRY_NAME='Russia', count=1),
 Row(ORIGIN_COUNTRY_NAME='Anguilla', count=1),
 Row(ORIGIN_COUNTRY_NAME='Senegal', count=1),
 Row(ORIGIN_COUNTRY_NAME='Sweden', count=1),
 Row(ORIGIN_COUNTRY_NAME='Kiribati', count=1),
 Row(ORIGIN_COUNTRY_NAME='Guyana', count=1),
 Row(ORIGIN_COUNTRY_NAME='Philippines', count=1),
 Row(ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(ORIGIN_COUNTRY_NAME='Malaysia', count=1),
 Row(ORIGIN_COUNTRY_NAME='Fiji', count=1),
 Row(ORIGIN_COUNTRY_NAME='Turkey', count=1),
 Row(ORIGIN_COUNTRY_NAME='Germany', count=1),
 Row(ORIGIN_COUNTRY_NAME='Jordan', count=1),
 Row(ORIGIN_COUNTRY_NAME='Palau', count=1),
 Row(ORIGIN_COUNTRY_NAME='Turks and Caicos Islands', count=1),
 Row(ORIGIN_COUNTRY_NAME='France', count=1),
 Row(ORIGIN_COUNTRY_NAME='Greece', count=1),
 Row(ORIGIN_COUNTRY_NAME='British Virgin Islands', count=1),
 Row(ORIGIN_COUNTRY_NAME='Taiwan', count=1),
 Row(ORIGIN_COUNTRY_NAME='Dominica', count=1),
 Row

In [9]:
%%time
DF4 = DF1.groupBy("count").count().collect()

CPU times: user 4.13 ms, sys: 1.41 ms, total: 5.54 ms
Wall time: 805 ms


In [17]:
# COMMAND ----------

DF1.cache()

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [18]:
%%time
DF1.count()

CPU times: user 1.44 ms, sys: 233 µs, total: 1.67 ms
Wall time: 290 ms


256

In [19]:
%%time
DF2 = DF1.groupBy("DEST_COUNTRY_NAME").count().collect()

CPU times: user 4.64 ms, sys: 3.58 ms, total: 8.22 ms
Wall time: 657 ms


In [20]:
%%time
DF3 = DF1.groupBy("ORIGIN_COUNTRY_NAME").count().collect()

CPU times: user 7 ms, sys: 0 ns, total: 7 ms
Wall time: 512 ms


In [22]:
DF4 = DF1.groupBy("count").count().collect()

In [23]:
DF4

[Row(count=31, count=1),
 Row(count=2025, count=1),
 Row(count=588, count=1),
 Row(count=53, count=1),
 Row(count=853, count=1),
 Row(count=362, count=1),
 Row(count=1468, count=1),
 Row(count=155, count=1),
 Row(count=108, count=1),
 Row(count=211, count=1),
 Row(count=34, count=1),
 Row(count=193, count=2),
 Row(count=126, count=2),
 Row(count=115, count=1),
 Row(count=772, count=1),
 Row(count=28, count=2),
 Row(count=183, count=1),
 Row(count=1496, count=1),
 Row(count=300, count=1),
 Row(count=332, count=1),
 Row(count=26, count=2),
 Row(count=27, count=1),
 Row(count=44, count=3),
 Row(count=236, count=1),
 Row(count=329, count=1),
 Row(count=12, count=5),
 Row(count=230, count=1),
 Row(count=225, count=1),
 Row(count=246, count=1),
 Row(count=346, count=1),
 Row(count=111, count=1),
 Row(count=8399, count=1),
 Row(count=370002, count=1),
 Row(count=827, count=1),
 Row(count=152, count=1),
 Row(count=185, count=1),
 Row(count=305, count=1),
 Row(count=325, count=2),
 Row(count=95

In [24]:
DF1.is_cached

True

In [25]:
DF1.cache?